In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [ ]:
# Load the dataset
data = pd.read_csv('Churn_Modelling.csv') 
data.head()

In [ ]:
## Preprocess the data

### Drop unnecessary columns
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [ ]:
## Apply Encoding to categorical variables
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [ ]:
data.Geography.unique()

In [ ]:
## Apply One-Hot Encoding to Geography
ohe_gepgraphy = OneHotEncoder(sparse_output=True)
geo_encoder = ohe_gepgraphy.fit_transform(data[['Geography']])
geo_encoder

In [ ]:
ohe_gepgraphy.get_feature_names_out(['Geography'])

In [ ]:
geo_encoder_df = pd.DataFrame(geo_encoder.toarray(), columns = ohe_gepgraphy.get_feature_names_out(['Geography']))

In [ ]:
## Combine the one hot encoded columns with original data
data = pd.concat([data.drop('Geography', axis=1), geo_encoder_df], axis=1)

In [ ]:
data.head()

In [ ]:

## Saving the encoders as pkl files
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('one_hot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(ohe_gepgraphy, file)

In [ ]:
## Divide the data into features and target variable
X = data.drop('Exited', axis=1)
y = data['Exited']

## Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Scale the featires (Standard Scaler)
scalar = StandardScaler()
X_train = scalar.fit_transform(X_train)
X_test = scalar.transform(X_test)



In [ ]:
## Saving the scaler as pkl file
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scalar, file)

## ANN Implementation

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime


In [ ]:
## Build ANN
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),  ## Hiddel layer 1 Connected with I/P layer
    Dense(32, activation='relu'),  ## Hidden layer 2
    Dense(1, activation='sigmoid')  ## Output layer (Sigmoid bcoz binary o/p)
]
)

In [ ]:
model.summary()

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001)  ## Adam optimizer with learning rate 0.001
loss = tf.keras.losses.BinaryCrossentropy()  ## Binary Crossentropy loss function

In [ ]:
## Compile the model
model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

In [ ]:
## Setup tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


In [ ]:
## Setup early stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
### Train the model
history = model.fit(
    X_train, y_train,validation_data=(X_test, y_test), epochs=100,
    callbacks=[early_stopping_callback, tensorflow_callback],
)

In [ ]:
model.save('model.h5')  ## Save the model

In [ ]:
## Load TensorBoard extention
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/fit

In [1]:
## Load the Pickle file
import tensorflow as tf
print(tf.__version__)


2.15.0
